# ECMAScript Exploraion, Revisit

## OUTLINE

- ### Symbol
- ### Iteration
- ### Protocol
- ### Generator
- ### Map
- ### Set
- ### More on Protocol

# `Symbol`

> ### ES6 symbols are values, but they’re not strings. They’re not objects. They’re something new: *a seventh type of value*.

### And these are the rest of six ofprimitive types of Javascript.

- `null`
- `undefined`
- `Boolean`
- `String`
- `Number`
- `Bigint`

In [1]:
typeof Symbol()

'symbol'

In [2]:
Symbol()

Symbol()

In [3]:
new Symbol();

TypeError: Symbol is not a constructor

## Why we need symbol ?

> ### Symbols are *unique* so They are good at being *keys* for preventing keyname collision.

In [4]:
// symbol is unique

var uniq = Symbol('uniq')  // create symbol with descriptor not a string.

uniq === Symbol('uniq');

false

In [5]:
// use symbol in global scope

var for_uniq = Symbol.for('for_uniq');  // registry in global scope.
for_uniq === Symbol.for('for_uniq');

true

In [6]:
// using symbol as key

var o = { prop1: 1, prop2: 2 };
o[uniq] = 'uniq';

console.log(o[uniq], o['uniq'], o[Symbol('uniq')])

uniq undefined undefined


### Symbols might be used as private properties.

In [7]:
Object.getOwnPropertyNames(o)

[ 'prop1', 'prop2' ]

In [8]:
// but they can retrive by

Object.getOwnPropertySymbols(o)  // no more private ?

[ Symbol(uniq) ]

### Is `Symbol` delcaration more  memory effient than just `String` ?

In [9]:
// helper for get memory heap of current process.
function getCurrentMem() { return process.memoryUsage().heapUsed / 1024 }

// trying to measure symbol mem usage after declar
var mem1 = getCurrentMem()
var newSymbol = Symbol();
getCurrentMem() - mem1;

0.921875

In [10]:
// and compare with simple string but it seems not work.
var mem = getCurrentMem();
var newSymbol2 = 'newSYmbol';
getCurrentMem() - mem;

0.2734375

## Iteration Object

---

> Iteration object is the object that can be looped over `for...of` construct.

In [11]:
// String is iteration object.

var iterString = 'String is iterable!';
for (s of iterString) 
    console.log(s);

S
t
r
i
n
g
 
i
s
 
i
t
e
r
a
b
l
e
!


In [12]:
// Array also is iteration object.

var iterArray = [1,2,3];
for (i of iterArray)
    console.log(i)

1
2
3


In [13]:
// Iteration object can be spreaded

[...iterString]

[
  'S', 't', 'r', 'i', 'n',
  'g', ' ', 'i', 's', ' ',
  'i', 't', 'e', 'r', 'a',
  'b', 'l', 'e', '!'
]

## How can those objects do iteration ? .... Protocol

----

> Protocol is a set of definitions to implement in order to conform with constructs.

## Iterable and Iterator Protocol

---

> To say an object is iterable means that object must implement the `iterable` protocol. The same case with `iterator`

### What is iterable protocol look like ?

> A property `Symbol.iterator` (`@@iterator`) which is a zero-args function that returns an object with `next` property return `done` and `value` as keys


In [14]:
var noneIter = {
    //    iterable                              iterator
    // |--- vvv ---------|   | ------------------- vvv --------------------------------|
    [Symbol.iterator]: () => ({ next: () => ({ done: true, value: 'never been sent.' })})
}

for (i of noneIter) 
    console.log('never reach but not error by for..of', i)

console.log('The iteration finish.')

The iteration finish.


In [16]:
var iterableOnly = {
    next: () => ({ done: true }),
    [Symbol.iterator]: function() { return this }
}

console.log('!!This will be error since next method not implemented.');
            
for (i of iterableOnly)
    console.log(i)

!!This will be error since next method not implemented.


In [17]:
// need to construct a String object explicitly to avoid auto-boxing
var reversedWords = new String("Good words are no words – who said no words.");

reversedWords[Symbol.iterator] = function() {
    return {
        next: function () {
            if (this._words.length)
                return { value: this._words.pop(), done: false }
            return { done: true }  // for loop will be stop by done value is true
        },
        _words: this.split(' ')  // keep original
    }
};

for (const word of reversedWords) {
    console.log(word)
}

words.
no
said
who
–
words
no
are
words
Good


In [18]:
[...reversedWords]

[
  'words.', 'no',
  'said',   'who',
  '–',      'words',
  'no',     'are',
  'words',  'Good'
]

In [19]:
// use `next`
var reversedIter = reversedWords[Symbol.iterator]()

console.log(1, reversedIter.next())
console.log(2, reversedIter.next().value)
console.log(3, reversedIter.next().done)
console.log(4, reversedIter.next())

1 { value: 'words.', done: false }
2 no
3 false
4 { value: 'who', done: false }


----

## `Generator` and `Generator function`

----

> `Generator` is object that generate any values which initiated by `Generator function`. Generator is both iterator and iterable

In [20]:
function* generatorFn() { yield '!!' }
var generator = generatorFn();

generator[Symbol.iterator]; // implemented iterable protocol.

[Function: [Symbol.iterator]]

In [21]:
// Yield can be used only inside generator
function* anyGen() {
    yield [1];
    yield 2;
    yield true;
    yield undefined;
    yield Symbol('generator!');
}

var gen = anyGen();

console.log(gen)
console.log(1, gen.next())
console.log(2, gen.next())
console.log(3, gen.next())
console.log(4, gen.next())
console.log(5, gen.next())
console.log('last', gen.next())

Object [Generator] {}
1 { value: [ 1 ], done: false }
2 { value: 2, done: false }
3 { value: true, done: false }
4 { value: undefined, done: false }
5 { value: Symbol(generator!), done: false }
last { value: undefined, done: true }


In [22]:
gen.next()
gen.next()

{ value: undefined, done: true }

In [23]:
for (item of anyGen()) {
    console.log(item)
}

[ 1 ]
2
true
undefined
Symbol(generator!)


In [24]:
[...anyGen()]

[ [ 1 ], 2, true, undefined, Symbol(generator!) ]

In [25]:
// revise reversedWord with generator

var reversedWords = new String("Good words are no words – who said no words.");

reversedWords[Symbol.iterator] = function* () {
    let _words = this.split(' ')
    while (_words.length)
        yield _words.pop();
};

for (const word of reversedWords) {
    console.log(word)
}

[...reversedWords].join(' ')

words.
no
said
who
–
words
no
are
words
Good


'words. no said who – words no are words Good'

In [26]:
// passing arguement in next

function* coGenerator () {
    let sum = 0
    let input;
    while (true) {
        input = yield sum
        if (input) sum += input
    }
}

var coGen = coGenerator()

console.log(coGen.next())
console.log(coGen.next(2))
console.log(coGen.next(10))
console.log(coGen.next())
console.log(coGen.next(1))

{ value: 0, done: false }
{ value: 2, done: false }
{ value: 12, done: false }
{ value: 12, done: false }
{ value: 13, done: false }


In [29]:
// Async / Await generator

async function* aGenerator() {
    // delay 1 sec
    await new Promise(resolve => setTimeout(resolve, 1500))
    
    yield 'ok!!'
}

(async () => {
    for await (i of aGenerator())
        console.log(i)
})()

ok!!


---

## Use of generators ?

---

> ### Memory efficient when dealing looping over milions of records from database or milions of lines in files since generator is lazy evaluation function.


> ### Infinite Stream of data.

---

## Who can eat up iterable object as initial argument ?

---

- `Map`
- `Set`
- `WeakMap`
- `WeakSet`
- `Promise.all`
- `Promise.race`
- `Array.from`

In [30]:
Array.from(reversedWords)

[
  'words.', 'no',
  'said',   'who',
  '–',      'words',
  'no',     'are',
  'words',  'Good'
]

In [31]:
function* initColorMap() {
    yield [Symbol(), 'RED']
    yield [Symbol(), 'GREEN']
    yield [Symbol(), 'BLUE']
}

new Map(initColorMap())

Map { Symbol() => 'RED', Symbol() => 'GREEN', Symbol() => 'BLUE' }

-----
Map
-----

> The Map object holds **key-value pairs** and remembers the original insertion **order** of the keys. Any value (both objects and primitive values) may be used as either a key or a value.


---
Map vs Object
---

### 1. Map start with no keys but Object not (storing prototype ...)

In [32]:
var aMap = new Map();
var o = {};

console.dir(aMap);
console.log(aMap.__proto__)
console.dir({});
console.dir(o.__proto__)

Map {}
Map {}
{}
{}


### 2. Map's keys can be any value but Object must be  either `String` or `Symbol`

In [33]:
var aMap = new Map()
var o = {}

aMap.set('string', 'string')
o['string'] = 'string'

aMap.set(1, 'integer')
o[1] = 'integer' // this will be coverted into string of '1'

aMap.set(1.1, 'floating')
o[1.1] = 'floating'

aMap.set(NaN, 'NaN NaN')
o[NaN] = 'Oh NaN'

var fn = function() {}

aMap.set(fn, 'function')
o[fn] = 'function'

aMap.set(Symbol(), 'symbol')
o[Symbol('random')] = 'symbol'; // It wont show because Symbol in object intend to be private

// table map
console.log('Map Keys')
console.table(aMap.keys())

// table object
console.log('Object Keys')
console.table(Object.keys(o))

// please notice order-ness

Map Keys
┌───────────────────┬────────────────┐
│ (iteration index) │     Values     │
├───────────────────┼────────────────┤
│         0         │    'string'    │
│         1         │       1        │
│         2         │      1.1       │
│         3         │      NaN       │
│         4         │ [Function: fn] │
│         5         │    Symbol()    │
└───────────────────┴────────────────┘
Object Keys
┌─────────┬─────────────────┐
│ (index) │     Values      │
├─────────┼─────────────────┤
│    0    │       '1'       │
│    1    │    'string'     │
│    2    │      '1.1'      │
│    3    │      'NaN'      │
│    4    │ 'function() {}' │
└─────────┴─────────────────┘


### 3. Getting Size

In [34]:
console.log('Size of Map', aMap.size);
console.log('Size of Object', Object.keys(o).length)

Size of Map 6
Size of Object 5


### 4. Key checking

In [35]:
console.log('NaN is in Map ?', aMap.has(NaN))
console.log('NaN is in Object ?', NaN in o);

NaN is in Map ? true
NaN is in Object ? true


### 5. Iteration

In [36]:
console.log('## Iterate over Map')
for ([k, v] of aMap)
    console.log(typeof k, k, v)

console.log('----------------------------')
console.log('## Iterate over Object')
for ([k, v] of Object.entries(o))
    console.log(typeof k, k, v)

## Iterate over Map
string string string
number 1 integer
number 1.1 floating
number NaN NaN NaN
function [Function: fn] function
symbol Symbol() symbol
----------------------------
## Iterate over Object
string 1 integer
string string string
string 1.1 floating
string NaN Oh NaN
string function() {} function


### 6. Delete a key

In [37]:
// delete from Map
aMap.delete(NaN)
console.table(aMap)

// delete from object
delete o[NaN]

console.table(o)

┌───────────────────┬────────────────┬────────────┐
│ (iteration index) │      Key       │   Values   │
├───────────────────┼────────────────┼────────────┤
│         0         │    'string'    │  'string'  │
│         1         │       1        │ 'integer'  │
│         2         │      1.1       │ 'floating' │
│         3         │ [Function: fn] │ 'function' │
│         4         │    Symbol()    │  'symbol'  │
└───────────────────┴────────────────┴────────────┘
┌───────────────┬────────────┐
│    (index)    │   Values   │
├───────────────┼────────────┤
│       1       │ 'integer'  │
│    string     │  'string'  │
│      1.1      │ 'floating' │
│ function() {} │ 'function' │
└───────────────┴────────────┘


### 7. Performance !

In [38]:
console.log('testing insert 1 mil of index and random');

var prefMap = new Map();
var t1 = new Date().getTime();

// 1. index => index
Array.from(Array(1e6)).forEach((v, i) => prefMap.set(i, i));
console.log("Map(index): \t", (new Date().getTime() - t1) + ' ms');
var prefMap = new Map();
var t1 = new Date().getTime();

// 2. random => random
Array.from(Array(1e6)).forEach((v, i) => prefMap.set("str" + i, Math.random()));
console.log("Map(random): \t", (new Date().getTime() - t1) + ' ms');
prefMap = null; // free mem
// -----------------------------------------------------------
var prefObj = {};
var t1 = new Date().getTime();

// 1. index => index
Array.from(Array(1e6)).forEach((v, i) => prefObj[i] = i);
console.log("OBJ(index): \t", (new Date().getTime() - t1) + ' ms');

var prefObj = {};
var t1 = new Date().getTime();
// 2. random => random
Array.from(Array(1e6)).forEach((v, i) => prefObj["str" + i] = Math.random() );

console.log("OBJ(random): \t", (new Date().getTime() - t1) + ' ms');

testing insert 1 mil of index and random
Map(index): 	 309 ms
Map(random): 	 788 ms
OBJ(index): 	 93 ms
OBJ(random): 	 1326 ms


---

## Set

----

> The Set object lets you store unique values of any type, whether primitive values or object references.

In [39]:
var aSet = new Set(reversedWords)
aSet

Set { 'words.', 'no', 'said', 'who', '–', 'words', 'are', 'Good' }

### What does stroring unique value mean for Set ?

In [40]:
var setB = new Set()

setB.add('orange');

Set { 'orange' }

In [41]:
// add another orange and apple.

setB.add('orange')
setB.add('apple')
setB

Set { 'orange', 'apple' }

In [42]:
// Set is iterable
setB[Symbol.iterator]

[Function: values]

In [43]:
for (item of setB)
    console.log(item)

orange
apple


## Trying implement `search` protocol

> What if set can accept search method from regexp `"orange".search(set)`

In [44]:
setB[Symbol.search] = function(string) {
    return this.has(string)
}

"orange".search(setB)

true

In [45]:
"nothing".search(setB)

false

In [46]:
// list all protocol we can play with

Object.getOwnPropertyNames(Symbol).filter(s => typeof Symbol[s] === 'symbol')

[
  'asyncIterator',
  'hasInstance',
  'isConcatSpreadable',
  'iterator',
  'match',
  'replace',
  'search',
  'species',
  'split',
  'toPrimitive',
  'toStringTag',
  'unscopables',
  'matchAll'
]

---


# Thank You  // Q & A  // Happy Friday !

----


----

## References

----

- https://stackoverflow.com/questions/21724326/what-is-the-motivation-for-bringing-symbols-to-es6
- https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Iteration_protocols
- https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Generator
- https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Map
- https://exploringjs.com/deep-js/toc.html
- https://tc39.es/
- https://en.wikipedia.org/wiki/ECMAScript
- https://www.valentinog.com/blog/node-usage/
- https://developer.mozilla.org/en-US/docs/Glossary/Primitive
- https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Symbol
- https://hacks.mozilla.org/2015/06/es6-in-depth-symbols/
- https://developer.mozilla.org/en-US/docs/Web/JavaScript/Memory_Management
- https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Global_Objects/Map